<a href="https://colab.research.google.com/github/Lakshika-Swarnamali/Diagrams/blob/master/FYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use ('fivethirtyeight')
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

In [ ]:
log = pd.read_csv("https://raw.githubusercontent.com/GihanKLG/TwitterCovid19Dataset/master/2020-10-15.csv",encoding='latin1')

log[(log['lang'] == 'en')].to_csv("out.csv", index=False)


log = pd.read_csv('out.csv', lineterminator='\n')

log.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Thu Oct 15 00:36:58 +0000 2020,NaN,NaN,NaN,0,1316538523521085441,NaN,NaN,NaN,en,NaN,NaN,1,1.316538e+18,realRonDeSantis,"<a href=""http://twitter.com/download/iphone"" r...",RT @realRonDeSantis: @acrimonyand @JayFiveK â...,https://twitter.com/acrimonyand/status/1316538...,Fri Oct 11 02:31:40 +0000 2019,acrimonyand,False,Colonial. Southerner. Texan. Dancehall Nationa...,38775,1268,680,6,"Marienfeld, Tx",E. H. Looney,acrimonyand,19600,NaN,https://anglicanorumcoetibussociety.blog,False
1,NaN,Thu Oct 15 00:36:59 +0000 2020,NaN,NaN,NaN,0,1316538525722927105,NaN,NaN,NaN,en,NaN,NaN,4873,1.316057e+18,CGreenUK,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @CGreenUK: The Bolton lockdown has clearly ...,https://twitter.com/PhHateLoveUK/status/131653...,Sun May 10 02:12:23 +0000 2015,PhHateLoveUK,False,Fighting On. Spit & Vinegar gets me through th...,23009,406,1447,1,Citizen Of Hell,Taken my Borrowed Vote back Boris,PhHateLoveUK,33201,NaN,NaN,False
2,NaN,Thu Oct 15 00:36:58 +0000 2020,NaN,NaN,NaN,0,1316538521767874568,NaN,NaN,NaN,en,NaN,NaN,2077,1.316525e+18,chrislhayes,"<a href=""http://twitter.com/download/android"" ...","RT @chrislhayes: Weâve gone from 30,000 hosp...",https://twitter.com/Ynot_12K/status/1316538521...,Sun Jun 08 03:19:43 +0000 2014,Ynot_12K,True,NaN,4358,50,576,3,NaN,Tony,Ynot_12K,65359,NaN,NaN,False
3,NaN,Thu Oct 15 00:36:58 +0000 2020,NaN,NaN,https://twitter.com/GunnelsWarren/status/13146...,8,1316538524447932416,NaN,NaN,NaN,en,NaN,False,4,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",It's time to tax the wealthy and corporations ...,https://twitter.com/pattiistweeting/status/131...,Wed Sep 06 02:56:51 +0000 2017,pattiistweeting,False,NaN,83270,1941,1342,8,the big ð,pattið»boo,pattiistweeting,64606,NaN,NaN,False
4,NaN,Thu Oct 15 00:36:58 +0000 2020,NaN,NaN,NaN,0,1316538521922879489,NaN,NaN,NaN,en,NaN,NaN,2006,1.316508e+18,tedlieu,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @tedlieu: A whopping 78% of Americans suppo...,https://twitter.com/bluefootsue/status/1316538...,Fri Feb 03 02:32:07 +0000 2017,bluefootsue,True,NaN,1392,5,69,0,NaN,Cynthia Hagstrom,bluefootsue,760,NaN,NaN,False


In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
import random
text_data = []
f = list(log['text'])
# all_tweets = list(tweets)
# all_tweets = all_tweets[np.logical_not(np.isnull(all_tweets))]
f = [x for x in f if str(x) != 'nan']
for line in f:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        print(tokens)
        text_data.append(tokens)

['SCREEN_NAME', 'child', 'center', '57,000', 'provider', 'increase', 'covid-19', 'provide', 'person']
['SCREEN_NAME', 'trumpâ\x80\x99s', 'rally', 'â\x80\x9ccovid', 'superspreader', 'eventsâ\x80\x9d', 'thousand', 'protester', 'marching', 'street', 'perfectâ\x80']
['SCREEN_NAME', 'university', 'miami', 'openly', 'admit', 'using', 'facial', 'recognition', 'surveillance', 'crack', 'student']
['SCREEN_NAME', 'itâ\x80\x99s', 'getting', 'worse', 'suspect', 'people', 'bore', 'insanity', 'inducing', 'unending', 'drumbeaâ\x80']
['another', 'covid', 'establishment', 'wonâ\x80\x99t']
['SCREEN_NAME', 'american', 'adopt', 'sweden', 'model', 'covid-19']
['preach']
['SCREEN_NAME', 'short', 'america']
['SCREEN_NAME', 'president', 'trump', 'supreme', 'court', 'complication', 'covid-19', 'scar', 'heartâ\x80']
['SCREEN_NAME', 'SCREEN_NAME', 'calling', 'government', 'school', 'business', 'people', 'strugglingâ\x80']
['SCREEN_NAME', 'SCREEN_NAME', 'better', 'hereâ\x80\x99s', 'hope', 'people', 'test', 'sympt

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.064*"SCREEN_NAME" + 0.015*"trump" + 0.014*"create" + 0.013*"record"')
(1, '0.079*"SCREEN_NAME" + 0.027*"trump" + 0.023*"covid" + 0.017*"covid-19"')
(2, '0.071*"SCREEN_NAME" + 0.029*"trump" + 0.021*"covid" + 0.010*"think"')
(3, '0.076*"SCREEN_NAME" + 0.013*"pandemic" + 0.010*"covid-19" + 0.008*"virus"')
(4, '0.106*"SCREEN_NAME" + 0.026*"covid" + 0.015*"trump" + 0.010*"covid-19"')


In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[]
[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]


In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.077*"SCREEN_NAME" + 0.034*"trump" + 0.024*"covid" + 0.012*"covid-19"')
(1, '0.089*"SCREEN_NAME" + 0.010*"covid19" + 0.008*"covid-19" + 0.008*"pandemic"')
(2, '0.078*"SCREEN_NAME" + 0.024*"covid" + 0.010*"covid-19" + 0.010*"trump"')


In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.052*"SCREEN_NAME" + 0.024*"covid-19" + 0.012*"coronavirus" + 0.012*"admit"')
(1, '0.104*"SCREEN_NAME" + 0.049*"covid" + 0.028*"trump" + 0.009*"covid19"')
(2, '0.091*"SCREEN_NAME" + 0.029*"covid" + 0.024*"breaking" + 0.023*"trump"')
(3, '0.065*"SCREEN_NAME" + 0.017*"trump" + 0.017*"record" + 0.017*"create"')
(4, '0.049*"SCREEN_NAME" + 0.029*"covid-19" + 0.021*"trump" + 0.015*"president"')
(5, '0.045*"SCREEN_NAME" + 0.013*"coronavirus" + 0.011*"shock" + 0.008*"president"')
(6, '0.144*"SCREEN_NAME" + 0.010*"mask" + 0.010*"pandemic" + 0.009*"america"')
(7, '0.066*"SCREEN_NAME" + 0.020*"covid-19" + 0.014*"wealthy" + 0.012*"positive"')
(8, '0.079*"SCREEN_NAME" + 0.043*"trump" + 0.020*"covid" + 0.012*"people"')
(9, '0.042*"SCREEN_NAME" + 0.013*"lockdown" + 0.010*"people" + 0.010*"president"')


In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=3048ec447c503bdf68157e19acce38e2f5b92773e6f70af7fb4869578df0bc85
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:471: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:471: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:471: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
